In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt


with open("ANDI_DATA/track_2_single_trajec (v{})_dataset".format(1),"rb") as f:
    data_v1 = pickle.load(f)
    
with open("ANDI_DATA/track_2_single_trajec (v{})_dataset".format(2),"rb") as f:
    data_v2 = pickle.load(f)

with open("ANDI_DATA/track_2_single_trajec (v{})_dataset".format(3),"rb") as f:
    data_v3 = pickle.load(f)
    
with open("ANDI_DATA/track_2_single_trajec (v{})_dataset".format(4),"rb") as f:
    data_v4 = pickle.load(f)
    
with open("ANDI_DATA/track_2_single_trajec_model_confinement_{}_dataset".format(1),"rb") as f:
    data_confined1 = pickle.load(f)
    
with open("ANDI_DATA/track_2_single_trajec_model_confinement_{}_dataset".format(2),"rb") as f:
    data_confined2 = pickle.load(f)

In [2]:
## confined_dataset

data_confined = [[],[],[],[],[],[],[]]

for data in [data_confined1,data_confined2]:
    for n in range(len(data['confinement'][0])):
        data_confined[0].append(data['confinement'][0][n])
        data_confined[1].append(data['confinement'][1][n])
        data_confined[2].append(data['confinement'][2][n])
        data_confined[3].append(data['confinement'][3][n])
        data_confined[4].append(data['confinement'][4][n])
        data_confined[5].append(data['confinement'][5][n])
        data_confined[6].append(data['confinement'][6][n])


In [3]:
def TMSD(traj, t_lags):
    ttt = np.zeros_like(t_lags, dtype= float)
    for idx, t in enumerate(t_lags): 
        for p in range(len(traj)-t):
            ttt[idx] += (traj[p]-traj[p+t])**2            
        ttt[idx] /= len(traj)-t    
    return ttt

def ta_msd(data_x,data_y):
    TA_MSD = []
    lag = len(data_x) 
    for i,s in enumerate(range(1,lag)):
        tamsd_s = []
        for t in range(len(data_x) - s):
            tamsd_s.append(((data_x[t+s] - data_x[t])**2) + ((data_y[t+s] - data_y[t])**2))
        TA_MSD.append(np.asarray(np.sum(tamsd_s)/len(tamsd_s)))

    return np.asarray(TA_MSD)

def correlation(x,y):
    
    T = int(len(x) - 2)
    
    cor = np.zeros(T)
    for t in range(T):
        dx_1 = x[t+1] - x[t]
        dx_2 = x[t+2] - x[t+1]
        
        dy_1 = y[t+1] - y[t]
        dy_2 = y[t+2] - y[t+1]
        
        cor[t] = (dx_1*dx_2 + dy_1*dy_2)
    
    return np.mean(cor)


def feature_v2(xtrace,ytrace):
    
    traces = np.sqrt(xtrace**2 + ytrace**2)
    
    mean = np.mean(traces)    
    ms = np.std(traces) 


   
    vtraces = np.diff(traces)
    
    v = np.abs(vtraces)
    vmean = np.mean(v) 
    vms = np.std(v) 

    
    corr = correlation(xtrace,ytrace)

    r = traces
    f1 = np.max(r) - np.min(r) 
    f2 = np.max(v) - np.min(v) 
    
       
    tlag = np.linspace(1, len(traces)-1, 9, dtype = 'int')
    
    msd = TMSD(traces, tlag)
    A = np.vstack([np.log(tlag), np.ones(len(np.log(tlag)))]).T
    nw1, nw0 = np.linalg.lstsq(A, np.log(msd), rcond=None)[0]


    feature = np.asarray([mean, ms, vmean, vms, np.exp(nw0)/4,f1,f2, corr]) 



    return feature



In [4]:
alpha_range = np.linspace(0,2.0,11)
alpha_range

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ])

In [5]:
# #  single_dataset

alpha_range = np.linspace(0,2.0,11)
alpha_range

single_data = {}

for a_num,a_r in enumerate(alpha_range[:-1]):
    a_r = np.round(a_r,2)
    for model, data in enumerate([data_v1,data_v2,data_v3,data_v4]):
        data_a_r = [[],[],[],[],[],[],[]]
        for n in range(len(data['single'][0])):
            alpha = np.mean(data['single'][3][n])
            if a_r <= alpha < alpha_range[int(a_num + 1)]:
                data_a_r[0].append(data['single'][0][n])
                data_a_r[1].append(data['single'][1][n])
                data_a_r[2].append(data['single'][2][n])
                data_a_r[3].append(data['single'][3][n])
                data_a_r[4].append(data['single'][4][n])
                data_a_r[5].append(data['single'][5][n])
                data_a_r[6].append(data['single'][6][n])
        single_data[(a_r,model)] = data_a_r


index = [(0,2500), (2500,3200),(3200,3600)]



single_dataset = {}
for index_num, index_range in enumerate(index):
    single_dataset1 = [[],[],[],[],[],[],[]]
    for a_num,a_r in enumerate(alpha_range[:-1]):
        a_r = np.round(a_r,2)
        for model, data in enumerate([data_v1,data_v2,data_v3,data_v4]):
            for n in range(index_range[0],index_range[1]):
                single_dataset1[0].append(single_data[(a_r,model)][0][n])
                single_dataset1[1].append(single_data[(a_r,model)][1][n])
                single_dataset1[2].append(single_data[(a_r,model)][2][n])
                single_dataset1[3].append(single_data[(a_r,model)][3][n])
                single_dataset1[4].append(single_data[(a_r,model)][4][n])
                single_dataset1[5].append(single_data[(a_r,model)][5][n])
                single_dataset1[6].append(single_data[(a_r,model)][6][n])
                
    single_dataset[index_num] = single_dataset1 

In [10]:
len(single_dataset[0][0])

100000

In [89]:
# single_data feature extraction

X_single = {}
Y_single = {}

case = 'single'
for index in range(3): # train, val, test
    
    X = {}
    Y = {}

    n_step = 0

    for tau in [10]:
        Y_tau = {}
        X_tau = {}
        for num, data in enumerate([single_dataset[index]]):

            X_case = {}
            Y_case = {}

            N_max = len(data[0])
                
            mapping_dimension = 200 - tau + 1
            X_n = np.zeros((N_max , mapping_dimension, 8))
            Y_alpha = np.zeros((N_max ,200))
            Y_k = np.zeros((N_max ,200))

            for n in range(N_max):
  
                    
                N_t = int(len(data[0][n]) - tau + 1) 
                for t in range(N_t):
                    x = data[0][n][t:t+tau] - data[0][n][t]
                    y = data[1][n][t:t+tau] - data[1][n][t]



                    X_n[n,t] = (np.abs(feature_v2(x,y)))

                for t in range(len(data[0][n])):
                    Y_alpha[n,t] = data[3][n][t]
                    Y_k[n,t] = data[4][n][t]



                n_step += 1
                if n_step % 1000 == 0:
                    print('progress_rate = {}'.format(n_step/(N_max)))


            X_case[case] = X_n
            Y_case[(case,'alpha')] = Y_alpha
            Y_case[(case,'k')] = Y_k

        X[tau] = X_case
        Y[tau] = Y_case
    
    X_single[index] = X
    Y_single[index] = Y    



progress_rate = 0.01
progress_rate = 0.02
progress_rate = 0.03
progress_rate = 0.04
progress_rate = 0.05
progress_rate = 0.06
progress_rate = 0.07
progress_rate = 0.08
progress_rate = 0.09
progress_rate = 0.1
progress_rate = 0.11
progress_rate = 0.12
progress_rate = 0.13
progress_rate = 0.14
progress_rate = 0.15
progress_rate = 0.16
progress_rate = 0.17
progress_rate = 0.18
progress_rate = 0.19
progress_rate = 0.2
progress_rate = 0.21
progress_rate = 0.22
progress_rate = 0.23
progress_rate = 0.24
progress_rate = 0.25
progress_rate = 0.26
progress_rate = 0.27
progress_rate = 0.28
progress_rate = 0.29
progress_rate = 0.3
progress_rate = 0.31
progress_rate = 0.32
progress_rate = 0.33
progress_rate = 0.34
progress_rate = 0.35
progress_rate = 0.36
progress_rate = 0.37
progress_rate = 0.38
progress_rate = 0.39
progress_rate = 0.4
progress_rate = 0.41
progress_rate = 0.42
progress_rate = 0.43
progress_rate = 0.44
progress_rate = 0.45
progress_rate = 0.46
progress_rate = 0.47
progress_rate = 0

In [97]:
X_single[2][10]['single'].shape

(16000, 191, 8)

In [94]:
# confined_data feature extraction

X_confined = {}
Y_confined = {}

case = 'confinement'


for index, data_number in enumerate([(0,84000), (84000,106000), (106000,120000)]): # train, val, test

    X = {}
    Y = {}
    
    
    n_step = 0

    for tau in [10]:
        Y_tau = {}
        X_tau = {}
       

        N_max = np.abs(data_number[1] - data_number[0])
                
        mapping_dimension = 200 - tau + 1
        X_n = np.zeros((N_max , mapping_dimension, 8))
        Y_alpha = np.zeros((N_max ,200))
        Y_k = np.zeros((N_max ,200))

        for n in range(data_number[0], data_number[1]):     
            
            N_t = int(len(data_confined[0][n]) - tau + 1) 
            for t in range(N_t):
                x = data_confined[0][n][t:t+tau] - data_confined[0][n][t]
                y = data_confined[1][n][t:t+tau] - data_confined[1][n][t]


            
                X_n[n_step,t] = (np.abs(feature_v2(x,y)))

            for t in range(len(data_confined[0][n]) ):
                Y_alpha[n_step,t] = data_confined[3][n][t]
                Y_k[n_step,t] = data_confined[4][n][t]



            n_step += 1
    
            if n_step % 1000 == 0:
                print('progress_rate = {}'.format(n_step/(N_max)))


        X_tau[case] = X_n
        Y_tau[(case,'alpha')] = Y_alpha
        Y_tau[(case,'k')] = Y_k

    
    X_confined[index] = X_tau
    Y_confined[index] = Y_tau    

progress_rate = 0.011904761904761904
progress_rate = 0.023809523809523808
progress_rate = 0.03571428571428571
progress_rate = 0.047619047619047616
progress_rate = 0.05952380952380952
progress_rate = 0.07142857142857142
progress_rate = 0.08333333333333333
progress_rate = 0.09523809523809523
progress_rate = 0.10714285714285714
progress_rate = 0.11904761904761904
progress_rate = 0.13095238095238096
progress_rate = 0.14285714285714285
progress_rate = 0.15476190476190477
progress_rate = 0.16666666666666666
progress_rate = 0.17857142857142858
progress_rate = 0.19047619047619047
progress_rate = 0.20238095238095238
progress_rate = 0.21428571428571427
progress_rate = 0.2261904761904762
progress_rate = 0.23809523809523808
progress_rate = 0.25
progress_rate = 0.2619047619047619
progress_rate = 0.27380952380952384
progress_rate = 0.2857142857142857
progress_rate = 0.2976190476190476
progress_rate = 0.30952380952380953
progress_rate = 0.32142857142857145
progress_rate = 0.3333333333333333
progress_

In [112]:
# multi, immobile, dimerization feature extraction


N_num = [(0,21000), (21000,26500), (26500,30000)] # train_index, val_index, test_index

cases = ['multi', 'immobile', 'dimerization']

X = {}
Y = {}


for index in range(3):# train, val, test
    

    X_index = {}
    Y_index = {}


    for tau in [10]:
        Y_tau = {}
        X_tau = {}
        
        for num, data in enumerate([data_v1,data_v2,data_v3,data_v4]):
            
            X_case = {}
            Y_case = {}

            for case in cases:
                n_step = 0
                print('data_v{}_{} starting..'.format(num+1, case))
#                 N_max = len(data_retype_single[case][0])

                N_max = np.abs(N_num[index][1] - N_num[index][0])
                
                mapping_dimension = 200 - tau + 1
                X_n = np.zeros((N_max , mapping_dimension, 8))
                Y_alpha = np.zeros((N_max ,200))
                Y_k = np.zeros((N_max ,200))

                for n in range(N_num[index][0], N_num[index][1]):
  
                    
                    N_t = int(len(data[case][0][n]) - tau + 1) 
                    for t in range(N_t):
                       
                        x = data[case][0][n][t:t+tau] - data[case][0][n][t]
                        y = data[case][1][n][t:t+tau] - data[case][1][n][t]



                        X_n[n_step,t] = (np.abs(feature_v2(x,y)))

                    for t in range(len(data[case][0][n])):
                        Y_alpha[n_step,t] = data[case][3][n][t]
                        Y_k[n_step,t] = data[case][4][n][t]
    


                    n_step += 1
                    if n_step % 1000 == 0:
                         print('progress_rate = {}'.format(n_step/(N_max)))


                X_case[case] = X_n
                Y_case[(case,'alpha')] = Y_alpha
                Y_case[(case,'k')] = Y_k

            X_tau[num] = X_case
            Y_tau[num] = Y_case
        X_index[tau] = X_tau
        Y_index[tau] = Y_tau
    
    X[index] = X_index
    Y[index] = Y_index


data_v1_multi starting..
progress_rate = 0.047619047619047616
progress_rate = 0.09523809523809523
progress_rate = 0.14285714285714285
progress_rate = 0.19047619047619047
progress_rate = 0.23809523809523808
progress_rate = 0.2857142857142857
progress_rate = 0.3333333333333333
progress_rate = 0.38095238095238093
progress_rate = 0.42857142857142855
progress_rate = 0.47619047619047616
progress_rate = 0.5238095238095238
progress_rate = 0.5714285714285714
progress_rate = 0.6190476190476191
progress_rate = 0.6666666666666666
progress_rate = 0.7142857142857143
progress_rate = 0.7619047619047619
progress_rate = 0.8095238095238095
progress_rate = 0.8571428571428571
progress_rate = 0.9047619047619048
progress_rate = 0.9523809523809523
progress_rate = 1.0
data_v1_immobile starting..
progress_rate = 0.047619047619047616
progress_rate = 0.09523809523809523
progress_rate = 0.14285714285714285
progress_rate = 0.19047619047619047
progress_rate = 0.23809523809523808
progress_rate = 0.2857142857142857
pr

progress_rate = 0.8571428571428571
progress_rate = 0.9047619047619048
progress_rate = 0.9523809523809523
progress_rate = 1.0
data_v4_dimerization starting..
progress_rate = 0.047619047619047616
progress_rate = 0.09523809523809523
progress_rate = 0.14285714285714285
progress_rate = 0.19047619047619047
progress_rate = 0.23809523809523808
progress_rate = 0.2857142857142857
progress_rate = 0.3333333333333333
progress_rate = 0.38095238095238093
progress_rate = 0.42857142857142855
progress_rate = 0.47619047619047616
progress_rate = 0.5238095238095238
progress_rate = 0.5714285714285714
progress_rate = 0.6190476190476191
progress_rate = 0.6666666666666666
progress_rate = 0.7142857142857143
progress_rate = 0.7619047619047619
progress_rate = 0.8095238095238095
progress_rate = 0.8571428571428571
progress_rate = 0.9047619047619048
progress_rate = 0.9523809523809523
progress_rate = 1.0
data_v1_multi starting..
progress_rate = 0.18181818181818182
progress_rate = 0.36363636363636365
progress_rate = 0

In [156]:
Y_single[0][10][('single','alpha')].shape




(100000, 200)

In [161]:
# train/val/test data generate

# input_data

x_single_train = X_single[0][10]['single']
x_single_val = X_single[1][10]['single']
x_single_test = X_single[2][10]['single']

x_confined_train = X_confined[0]['confinement']
x_confined_val = X_confined[1]['confinement']
x_confined_test = X_confined[2]['confinement']

x_multi_train = np.concatenate([X[0][10][0]['multi'], X[0][10][1]['multi'], X[0][10][2]['multi'], X[0][10][3]['multi']])
x_multi_val = np.concatenate([X[1][10][0]['multi'], X[1][10][1]['multi'], X[1][10][2]['multi'], X[1][10][3]['multi']])
x_multi_test = np.concatenate([X[2][10][0]['multi'], X[2][10][1]['multi'], X[2][10][2]['multi'], X[2][10][3]['multi']])

x_dimerization_train = np.concatenate([X[0][10][0]['dimerization'], X[0][10][1]['dimerization'], X[0][10][2]['dimerization'], X[0][10][3]['dimerization']])
x_dimerization_val = np.concatenate([X[1][10][0]['dimerization'], X[1][10][1]['dimerization'], X[1][10][2]['dimerization'], X[1][10][3]['dimerization']])
x_dimerization_test = np.concatenate([X[2][10][0]['dimerization'], X[2][10][1]['dimerization'], X[2][10][2]['dimerization'], X[2][10][3]['dimerization']])

x_immobile_train = np.concatenate([X[0][10][0]['immobile'], X[0][10][1]['immobile'], X[0][10][2]['immobile'], X[0][10][3]['immobile']])
x_immobile_val = np.concatenate([X[1][10][0]['immobile'], X[1][10][1]['immobile'], X[1][10][2]['immobile'], X[1][10][3]['immobile']])
x_immobile_test = np.concatenate([X[2][10][0]['immobile'], X[2][10][1]['immobile'], X[2][10][2]['immobile'], X[2][10][3]['immobile']])


# output_data (alpha,k)

alpha_single_train = Y_single[0][10][('single','alpha')]
alpha_single_val = Y_single[1][10][('single','alpha')]
alpha_single_test = Y_single[2][10][('single','alpha')]

alpha_confined_train = Y_confined[0][('confinement','alpha')]
alpha_confined_val = Y_confined[1][('confinement','alpha')]
alpha_confined_test = Y_confined[2][('confinement','alpha')]

alpha_multi_train = np.concatenate([Y[0][10][0][('multi','alpha')], Y[0][10][1][('multi','alpha')], Y[0][10][2][('multi','alpha')], Y[0][10][3][('multi','alpha')]])
alpha_multi_val = np.concatenate([Y[1][10][0][('multi','alpha')], Y[1][10][1][('multi','alpha')], Y[1][10][2][('multi','alpha')], Y[1][10][3][('multi','alpha')]])
alpha_multi_test = np.concatenate([Y[2][10][0][('multi','alpha')], Y[2][10][1][('multi','alpha')], Y[2][10][2][('multi','alpha')], Y[2][10][3][('multi','alpha')]])

alpha_dimerization_train = np.concatenate([Y[0][10][0][('dimerization','alpha')], Y[0][10][1][('dimerization','alpha')], Y[0][10][2][('dimerization','alpha')], Y[0][10][3][('dimerization','alpha')]])
alpha_dimerization_val = np.concatenate([Y[1][10][0][('dimerization','alpha')], Y[1][10][1][('dimerization','alpha')], Y[1][10][2][('dimerization','alpha')], Y[1][10][3][('dimerization','alpha')]])
alpha_dimerization_test = np.concatenate([Y[2][10][0][('dimerization','alpha')], Y[2][10][1][('dimerization','alpha')], Y[2][10][2][('dimerization','alpha')], Y[2][10][3][('dimerization','alpha')]])

alpha_immobile_train = np.concatenate([Y[0][10][0][('immobile','alpha')], Y[0][10][1][('immobile','alpha')], Y[0][10][2][('immobile','alpha')], Y[0][10][3][('immobile','alpha')]])
alpha_immobile_val = np.concatenate([Y[1][10][0][('immobile','alpha')], Y[1][10][1][('immobile','alpha')], Y[1][10][2][('immobile','alpha')], Y[1][10][3][('immobile','alpha')]])
alpha_immobile_test = np.concatenate([Y[2][10][0][('immobile','alpha')], Y[2][10][1][('immobile','alpha')], Y[2][10][2][('immobile','alpha')], Y[2][10][3][('immobile','alpha')]])



k_single_train = Y_single[0][10][('single','k')]
k_single_val = Y_single[1][10][('single','k')]
k_single_test = Y_single[2][10][('single','k')]

k_confined_train = Y_confined[0][('confinement','k')]
k_confined_val = Y_confined[1][('confinement','k')]
k_confined_test = Y_confined[2][('confinement','k')]

k_multi_train = np.concatenate([Y[0][10][0][('multi','k')], Y[0][10][1][('multi','k')], Y[0][10][2][('multi','k')], Y[0][10][3][('multi','k')]])
k_multi_val = np.concatenate([Y[1][10][0][('multi','k')], Y[1][10][1][('multi','k')], Y[1][10][2][('multi','k')], Y[1][10][3][('multi','k')]])
k_multi_test = np.concatenate([Y[2][10][0][('multi','k')], Y[2][10][1][('multi','k')], Y[2][10][2][('multi','k')], Y[2][10][3][('multi','k')]])

k_dimerization_train = np.concatenate([Y[0][10][0][('dimerization','k')], Y[0][10][1][('dimerization','k')], Y[0][10][2][('dimerization','k')], Y[0][10][3][('dimerization','k')]])
k_dimerization_val = np.concatenate([Y[1][10][0][('dimerization','k')], Y[1][10][1][('dimerization','k')], Y[1][10][2][('dimerization','k')], Y[1][10][3][('dimerization','k')]])
k_dimerization_test = np.concatenate([Y[2][10][0][('dimerization','k')], Y[2][10][1][('dimerization','k')], Y[2][10][2][('dimerization','k')], Y[2][10][3][('dimerization','k')]])

k_immobile_train = np.concatenate([Y[0][10][0][('immobile','k')], Y[0][10][1][('immobile','k')], Y[0][10][2][('immobile','k')], Y[0][10][3][('immobile','k')]])
k_immobile_val = np.concatenate([Y[1][10][0][('immobile','k')], Y[1][10][1][('immobile','k')], Y[1][10][2][('immobile','k')], Y[1][10][3][('immobile','k')]])
k_immobile_test = np.concatenate([Y[2][10][0][('immobile','k')], Y[2][10][1][('immobile','k')], Y[2][10][2][('immobile','k')], Y[2][10][3][('immobile','k')]])




train_dataset = {}
val_dataset = {}
test_dataset = {}
                                                                                                                                             
                                                                                                                                             
train_dataset['input_data'] = np.concatenate([x_multi_train,x_dimerization_train,x_immobile_train,x_single_train,x_confined_train])
val_dataset['input_data'] = np.concatenate([x_multi_val,x_dimerization_val,x_immobile_val,x_single_val,x_confined_val])
test_dataset['input_data'] = np.concatenate([x_multi_test,x_dimerization_test,x_immobile_test,x_single_test,x_confined_test])

train_dataset['alpha'] = np.concatenate([alpha_multi_train,alpha_dimerization_train,alpha_immobile_train,alpha_single_train,alpha_confined_train])
val_dataset['alpha'] = np.concatenate([alpha_multi_val,alpha_dimerization_val,alpha_immobile_val,alpha_single_val,alpha_confined_val])
test_dataset['alpha'] = np.concatenate([alpha_multi_test,alpha_dimerization_test,alpha_immobile_test,alpha_single_test,alpha_confined_test])

train_dataset['k'] = np.concatenate([k_multi_train,k_dimerization_train,k_immobile_train,k_single_train,k_confined_train])
val_dataset['k'] = np.concatenate([k_multi_val,k_dimerization_val,k_immobile_val,k_single_val,k_confined_val])
test_dataset['k'] = np.concatenate([k_multi_test,k_dimerization_test,k_immobile_test,k_single_test,k_confined_test])

In [168]:
print('train_dataset input shape = {}'.format(train_dataset['input_data'].shape))
print('val_dataset input shape = {}'.format(val_dataset['input_data'].shape))
print('test_dataset input shape = {}'.format(test_dataset['input_data'].shape))

print('train_dataset alpha shape = {}'.format(train_dataset['alpha'].shape))
print('val_dataset alpha shape = {}'.format(val_dataset['alpha'].shape))
print('test_dataset alpha shape = {}'.format(test_dataset['alpha'].shape))

print('train_dataset k shape = {}'.format(train_dataset['k'].shape))
print('val_dataset k shape = {}'.format(val_dataset['k'].shape))
print('test_dataset k shape = {}'.format(test_dataset['k'].shape))

train_dataset input shape = (436000, 191, 8)
val_dataset input shape = (116000, 191, 8)
test_dataset input shape = (72000, 191, 8)
train_dataset alpha shape = (436000, 200)
val_dataset alpha shape = (116000, 200)
test_dataset alpha shape = (72000, 200)
train_dataset k shape = (436000, 200)
val_dataset k shape = (116000, 200)
test_dataset k shape = (72000, 200)


In [169]:
with open('ANDI_DATA/train_dataset', 'wb') as f:
    pickle.dump(train_dataset, f)
    
with open('ANDI_DATA/val_dataset', 'wb') as f:
    pickle.dump(val_dataset, f)

with open('ANDI_DATA/test_dataset', 'wb') as f:
    pickle.dump(test_dataset, f)

In [170]:
with open('ANDI_DATA/train_dataset', 'rb') as f:
    train_dataset = pickle.load(f)
    
with open('ANDI_DATA/val_dataset', 'rb') as f:
    val_dataset = pickle.load(f)

with open('ANDI_DATA/test_dataset', 'rb') as f:
    test_dataset = pickle.load(f)

In [171]:
print('train_dataset input shape = {}'.format(train_dataset['input_data'].shape))
print('val_dataset input shape = {}'.format(val_dataset['input_data'].shape))
print('test_dataset input shape = {}'.format(test_dataset['input_data'].shape))

print('train_dataset alpha shape = {}'.format(train_dataset['alpha'].shape))
print('val_dataset alpha shape = {}'.format(val_dataset['alpha'].shape))
print('test_dataset alpha shape = {}'.format(test_dataset['alpha'].shape))

print('train_dataset k shape = {}'.format(train_dataset['k'].shape))
print('val_dataset k shape = {}'.format(val_dataset['k'].shape))
print('test_dataset k shape = {}'.format(test_dataset['k'].shape))

train_dataset input shape = (436000, 191, 8)
val_dataset input shape = (116000, 191, 8)
test_dataset input shape = (72000, 191, 8)
train_dataset alpha shape = (436000, 200)
val_dataset alpha shape = (116000, 200)
test_dataset alpha shape = (72000, 200)
train_dataset k shape = (436000, 200)
val_dataset k shape = (116000, 200)
test_dataset k shape = (72000, 200)
